# Geolocalización de ciudades de origen de migrantes

+ Api de Google Maps
+ Geoencoding, y ubicación de cada una en el mapa 

In [9]:
import pandas as pd
import numpy as np 
from matplotlib import pyplot as plt

%matplotlib inline

In [11]:
import requests

API_KEY = ''
GOOGLE_MAPS_API_URL = 'http://maps.googleapis.com/maps/api/geocode/json'


df_ciudades = pd.DataFrame(columns=['LUGNAC', 'LAT', 'LNG', 'PAIS', 'CIUDAD'])

noEncontradas = []
contador = 0

if contador < 1:
    for ciudad in open("ciudades.txt"):
        print(ciudad)
        
        geocode_url = GOOGLE_MAPS_API_URL + '?address={}&key{}'.format(ciudad, API_KEY)
        
        req = requests.get(geocode_url)
        res = req.json()

        # Si se me acabó la cuota de requestes: break
        print("En la peticion {}, el status es {}".format(contador, res['status']))
  
        if res['status'] == "OVER_QUERY_LIMIT":
            print("no me contesta google")
            contador = contador + 1
            break

        # Si no hay resultados para la buqueda: continue
        if len(res['results']) == 0:
            noEncontradas.append(ciudad)
            contador = contador + 1
        
        else:

            result = res['results'][0]

            def returned_component(criteria, components):
                for component in components:
                    if criteria in component["types"]:
                        return component["long_name"]

            geodata = pd.DataFrame(
                {
                    'LUGNAC':ciudad, 
                    'LAT':result['geometry']['location']['lat'],
                    'LNG':result['geometry']['location']['lng'],
                    'PAIS':lambda x: returned_component("country", result["address_components"]), 
                    'CIUDAD':lambda x: returned_component("locality", result["address_components"])
                }
            )

            df_ciudades.append(geodata)
            print(geodata)

            contador = contador + 1
            
print(df_ciudades)
print(noEncontradas)
print(contador)

 HADJIN

En la peticion 0, el status es OVER_QUERY_LIMIT
no me contesta google
Empty DataFrame
Columns: [LUGNAC, LAT, LNG, PAIS, CIUDAD]
Index: []
[]
1


In [ ]:
noEncontradas

In [ ]:
df_ciudades